# MC2-Project-1 Build a Market Simulator

In [1]:
# http://quantsoftware.gatech.edu/MC2-Project-1

from DataHelper import DataHelper

In [ ]:
# input: orders file (name of file), initial cash
# output: history of value

#  *** ASSUMPTIONS ***
# if we sell a symbol that we don't own, then we are shorting
# orders are filled with adj. close prices

# psuedo algorithm to process orders file:

# step 1 - create Prices dataFrame 
#    read in data file, what's earliest date, last date, call get data
#    make dataframe with date, [stock symbols], cash
#    cash is equal to 1.0 for each day (when dealing with foreign currencies could be other value)

# step 2 - create a trades dataFrame to log change of shares in stocks
#    date, [stock symbols that holds volume], cash (start at 0)
#    initialize with 0 and populate step by step 

# step 3 - populate trades dataFrame
#    step through orders file line by line and process (is just logging, doesn't total items)
#    will need for loop

# step 4 - create a holdings dataFrame
#    date, [stock symbols], cash
#    initialize everything to 0 and 1st day with initial cash
#    loop through trades dataFrame and populate each day with any trades + previous day volume

# step 5 - create a Values dataFrame
#    represents the value of dollars of each asset we hold
#    Values = holdings * price dataFrame
#
# step 6 - sum (axis=1) of Values of dataFrame to get daily value;


### Leverage

### ${ Leverage = \frac{\sum |investements|}{\sum investments  + cash} }$

sum (abs (investments)) / sum(investments) + cash
